In [3]:
# kaggle score: 0.78468

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

In [4]:
train_data = pd.read_csv('datasets/train.csv')
test_data = pd.read_csv('datasets/test.csv')

In [5]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [36]:
# Basic feature engineering
# https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/

import string

def substrings_in_string(big_string, substrings):
    for substring in substrings:
        big_string = str(big_string)
        if big_string.find(substring) != -1:
            return substring
    return np.nan

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
            'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
            'Don', 'Jonkheer']

# Add titles column to dataset
train_data['Title'] = train_data['Name'].map(lambda x: substrings_in_string(x, title_list))
test_data['Title'] = test_data['Name'].map(lambda x: substrings_in_string(x, title_list))

# Add deck column to dataset
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Default']
train_data['Deck'] = train_data['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
test_data['Deck'] = test_data['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

# Add FamilySize column to dataset
train_data['Family_Size'] = train_data['SibSp'] + train_data['Parch']
test_data['Family_Size'] = test_data['SibSp'] + test_data['Parch']

train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Deck,Family_Size
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,NaN,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,C,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Rev,NaN,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss,B,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Miss,NaN,3
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr,C,0


In [37]:
x = train_data.iloc[:, 2:].drop(['Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Deck'], axis=1).interpolate(method='linear', limit_direction='forward').values
y = train_data.iloc[:, 1].values

x

array([[3, 'male', 22.0, 'Mr', 1],
       [1, 'female', 38.0, 'Mrs', 1],
       [3, 'female', 26.0, 'Miss', 0],
       ...,
       [3, 'female', 22.5, 'Miss', 3],
       [1, 'male', 26.0, 'Mr', 0],
       [3, 'male', 32.0, 'Mr', 0]], dtype=object)

In [38]:
x_val = test_data.iloc[:, 1:].drop(['Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Deck'], axis=1).interpolate(method='linear', limit_direction='forward').values

x_val_ids = test_data.iloc[:, 0]

x_val

array([[3, 'male', 34.5, 'Mr', 0],
       [3, 'female', 47.0, 'Mrs', 1],
       [2, 'male', 62.0, 'Mr', 0],
       ...,
       [3, 'male', 38.5, 'Mr', 0],
       [3, 'male', 38.5, 'Mr', 0],
       [3, 'male', 38.5, 'Master', 2]], dtype=object)

In [39]:
# encode sex labels

le = LabelEncoder()
x[:, 1] = le.fit_transform(x[:, 1])
x_val[:, 1] = le.transform(x_val[:, 1])

le2 = LabelEncoder()
x[:, 3] = le2.fit_transform(x[:, 3])
x_val[:, 3] = le2.transform(x_val[:, 3])

# le3 = LabelEncoder()
# x[:, 4] = le3.fit_transform(x[:, 4])
# x_val[:, 4] = le3.transform(x_val[:, 4])

x

array([[3, 1, 22.0, 11, 1],
       [1, 0, 38.0, 12, 1],
       [3, 0, 26.0, 8, 0],
       ...,
       [3, 0, 22.5, 8, 3],
       [1, 1, 26.0, 11, 0],
       [3, 1, 32.0, 11, 0]], dtype=object)

In [40]:
# scale ages
sc = StandardScaler()

x[:, 2:4] = sc.fit_transform(x[:, 2:4])
x_val[:, 2:4] = sc.transform(x_val[:, 2:4])

In [41]:
# one hot encode travel classes for input vector
onehot_enc = OneHotEncoder(sparse=False)

temp_array = np.zeros((x.shape[0], x.shape[1] + 2))

temp_array[:, 3:] = x[:, 1:]

c = np.expand_dims(x[:, 0], axis=1)

c = onehot_enc.fit_transform(c)
temp_array[:, 0:3] = c

x = temp_array
x

array([[ 0.        ,  0.        ,  1.        , ..., -0.55604973,
         0.41873663,  1.        ],
       [ 1.        ,  0.        ,  0.        , ...,  0.59548094,
         0.97809845,  1.        ],
       [ 0.        ,  0.        ,  1.        , ..., -0.26816707,
        -1.25934884,  0.        ],
       ...,
       [ 0.        ,  0.        ,  1.        , ..., -0.5200644 ,
        -1.25934884,  3.        ],
       [ 1.        ,  0.        ,  0.        , ..., -0.26816707,
         0.41873663,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.16365693,
         0.41873663,  0.        ]])

In [42]:
# one hot encode travel classes for validation vector
temp_array = np.zeros((x_val.shape[0], x_val.shape[1] + 2))

temp_array[:, 3:] = x_val[:, 1:]

c = np.expand_dims(x_val[:, 0], axis=1)

c = onehot_enc.transform(c)
temp_array[:, 0:3] = c

x_val = temp_array
x_val

array([[ 0.        ,  0.        ,  1.        , ...,  0.3435836 ,
         0.41873663,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  1.24321694,
         0.97809845,  1.        ],
       [ 0.        ,  1.        ,  0.        , ...,  2.32277694,
         0.41873663,  0.        ],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.63146627,
         0.41873663,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.63146627,
         0.41873663,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.63146627,
        -1.81871066,  2.        ]])

In [43]:
# split x input vector to train/test sets

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 0)

x_train.shape

(712, 7)

In [44]:
# network architecture

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import metrics

model = Sequential()

model.add(Dense(units=7, activation='relu', kernel_initializer='random_uniform'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=14, activation='relu', kernel_initializer='random_uniform'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=1, activation='sigmoid', kernel_initializer='random_uniform'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[metrics.mae, metrics.binary_accuracy])

In [45]:
model.fit(x_train, y_train, batch_size=1, epochs=20)

Epoch 1/20
712/712 [==============================] - 2s 3ms/step - loss: 0.6289 - mean_absolute_error: 0.4483 - binary_accuracy: 0.6250
Epoch 2/20
712/712 [==============================] - 2s 2ms/step - loss: 0.5348 - mean_absolute_error: 0.3704 - binary_accuracy: 0.7528
Epoch 3/20
712/712 [==============================] - 2s 2ms/step - loss: 0.5074 - mean_absolute_error: 0.3497 - binary_accuracy: 0.7978
Epoch 4/20
712/712 [==============================] - 2s 2ms/step - loss: 0.4936 - mean_absolute_error: 0.3324 - binary_accuracy: 0.8062
Epoch 5/20
712/712 [==============================] - 2s 2ms/step - loss: 0.4749 - mean_absolute_error: 0.3195 - binary_accuracy: 0.7949
Epoch 6/20
712/712 [==============================] - 2s 2ms/step - loss: 0.4709 - mean_absolute_error: 0.3102 - binary_accuracy: 0.8160
Epoch 7/20
712/712 [==============================] - 2s 2ms/step - loss: 0.4600 - mean_absolute_error: 0.3012 - binary_accuracy: 0.8048
Epoch 8/20
712/712 [=====================

In [46]:
# validate model against unknown data

unknown_predictions = model.predict(x_val)

In [47]:
# format predictions to be binary rather than boolean or floats

unknown_predictions = [(x[0] >= 0.5) * 1 for x in unknown_predictions]
unknown_predictions

[0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [48]:
import csv

rows = []

for pid, pred in zip(x_val_ids, unknown_predictions):
    rows.append((pid, pred))

with open('output/new_beginings4.csv', 'w', newline='') as res:
    writer = csv.writer(res)
    writer.writerow(('PassengerId', 'Survived'))
    for row in rows:     
        writer.writerow(row)